# 🥇 Gold Layer — Aggregation & KPI Notebook
## Notebook: 03_gold_aggregation

**What this notebook does:**
- Reads clean validated data from Silver Delta table
- Produces three Gold tables optimised for analytics
- gold_city_rankings  → city level pollutant KPIs
- gold_pollutant_trends → daily trends per city
- gold_aqi_summary   → AQI category breakdown per city

**Run after:** `02_silver_transformation`
**Run before:** `04_visualization`

In [0]:
# ============================================================
# RELOAD CONFIGURATION
# ============================================================

DATABASE_NAME = "air_quality_db"

SILVER_TABLE     = f"{DATABASE_NAME}.silver_clean_measurements"
GOLD_TABLE_CITY  = f"{DATABASE_NAME}.gold_city_rankings"
GOLD_TABLE_TREND = f"{DATABASE_NAME}.gold_pollutant_trends"
GOLD_TABLE_AQI   = f"{DATABASE_NAME}.gold_aqi_summary"

spark.sql(f"USE {DATABASE_NAME}")

print("✅ Configuration reloaded.")

✅ Configuration reloaded.


In [0]:
# ============================================================
# READ FROM SILVER TABLE
# Only use valid records for Gold aggregations
# ============================================================

silver_df = spark.sql(f"""
    SELECT *
    FROM {SILVER_TABLE}
    WHERE is_valid = true
""")

silver_count = silver_df.count()

print(f"✅ Silver valid records loaded : {silver_count}")
print(f"\n📊 Records per city:")
spark.sql(f"""
    SELECT city, COUNT(*) as records
    FROM {SILVER_TABLE}
    WHERE is_valid = true
    GROUP BY city
    ORDER BY records DESC
""").show(25, truncate=False)

✅ Silver valid records loaded : 7033

📊 Records per city:
+-----------+-------+
|city       |records|
+-----------+-------+
|Lima       |1500   |
|Mumbai     |1024   |
|Mexico City|562    |
|London     |441    |
|Tokyo      |400    |
|Shanghai   |400    |
|Seoul      |400    |
|Beijing    |350    |
|Karachi    |250    |
|Bangkok    |250    |
|Nairobi    |250    |
|Lahore     |200    |
|New York   |200    |
|Paris      |153    |
|Lagos      |150    |
|Los Angeles|150    |
|Dhaka      |149    |
|Delhi      |100    |
|Jakarta    |54     |
|Cairo      |50     |
+-----------+-------+



In [0]:
# ============================================================
# GOLD TABLE 1 — CITY RANKINGS
# Aggregated pollutant stats per city
# Dominant AQI = most frequently occurring AQI category
# ============================================================

from pyspark.sql.functions import (
    col, avg, max, min, count,
    current_timestamp, round as spark_round,
    first
)

# Step 1 — Base aggregation per city + pollutant
city_rankings_df = spark.sql(f"""
    SELECT
        city,
        country,
        pollutant,
        ROUND(AVG(value), 2)    AS avg_value,
        ROUND(MAX(value), 2)    AS max_value,
        ROUND(MIN(value), 2)    AS min_value,
        COUNT(*)                AS reading_count,
        CURRENT_TIMESTAMP()     AS last_updated
    FROM {SILVER_TABLE}
    WHERE is_valid = true
    GROUP BY city, country, pollutant
    ORDER BY city, pollutant
""")

# Step 2 — Find dominant AQI category per city (PM2.5 only)
dominant_aqi_df = spark.sql(f"""
    SELECT
        city,
        aqi_category AS dominant_aqi
    FROM (
        SELECT
            city,
            aqi_category,
            COUNT(*) AS cnt,
            ROW_NUMBER() OVER (PARTITION BY city ORDER BY COUNT(*) DESC) AS rn
        FROM {SILVER_TABLE}
        WHERE pollutant  = 'pm25'
        AND   is_valid   = true
        AND   aqi_category != 'N/A'
        GROUP BY city, aqi_category
    )
    WHERE rn = 1
""")

# Step 3 — Join dominant AQI onto city rankings
city_rankings_final_df = city_rankings_df.join(
    dominant_aqi_df, on="city", how="left"
)

# city_rankings_final_df.cache()

print(f"✅ City rankings aggregation complete.")
print(f"   Rows generated : {city_rankings_final_df.count()}")
print(f"\n📊 Preview — PM2.5 City Rankings:")
city_rankings_final_df \
    .filter(col("pollutant") == "pm25") \
    .select("city", "pollutant", "avg_value", "max_value", "min_value", "reading_count", "dominant_aqi") \
    .orderBy("avg_value", ascending=False) \
    .show(25, truncate=False)

✅ City rankings aggregation complete.
   Rows generated : 68

📊 Preview — PM2.5 City Rankings:
+-----------+---------+---------+---------+---------+-------------+------------+
|city       |pollutant|avg_value|max_value|min_value|reading_count|dominant_aqi|
+-----------+---------+---------+---------+---------+-------------+------------+
|Delhi      |pm25     |267.44   |439.0    |93.0     |50           |Hazardous   |
|Dhaka      |pm25     |121.64   |299.9    |47.7     |100          |Unhealthy   |
|Lahore     |pm25     |120.05   |329.97   |0.43     |150          |Moderate    |
|Cairo      |pm25     |63.46    |118.0    |23.0     |50           |Unhealthy   |
|Karachi    |pm25     |48.7     |101.4    |13.91    |150          |Moderate    |
|Lagos      |pm25     |45.42    |358.1    |0.0      |150          |Moderate    |
|Mumbai     |pm25     |36.14    |98.0     |1.0      |194          |Moderate    |
|Mexico City|pm25     |29.98    |77.0     |7.0      |52           |Moderate    |
|Shanghai   |p

In [0]:
# ============================================================
# GOLD TABLE 2 — POLLUTANT TRENDS
# Daily average readings per city per pollutant
# Used for time series trend visualisation
# ============================================================

trends_df = spark.sql(f"""
    SELECT
        city,
        pollutant,
        DATE(measured_at)       AS reading_date,
        ROUND(AVG(value), 2)    AS avg_value,
        COUNT(*)                AS reading_count
    FROM {SILVER_TABLE}
    WHERE is_valid      = true
    AND   measured_at   IS NOT NULL
    GROUP BY city, pollutant, DATE(measured_at)
    ORDER BY city, pollutant, reading_date
""")

#trends_df.cache()

print(f"✅ Pollutant trends aggregation complete.")
print(f"   Rows generated : {trends_df.count()}")
print(f"\n📊 Preview — Daily PM2.5 Trends for Delhi:")
trends_df \
    .filter((col("city") == "Delhi") & (col("pollutant") == "pm25")) \
    .show(10, truncate=False)

✅ Pollutant trends aggregation complete.
   Rows generated : 500

📊 Preview — Daily PM2.5 Trends for Delhi:
+-----+---------+------------+---------+-------------+
|city |pollutant|reading_date|avg_value|reading_count|
+-----+---------+------------+---------+-------------+
|Delhi|pm25     |2016-11-02  |300.0    |1            |
|Delhi|pm25     |2016-11-03  |249.55   |44           |
|Delhi|pm25     |2016-11-04  |418.4    |5            |
+-----+---------+------------+---------+-------------+



In [0]:
# ============================================================
# GOLD TABLE 3 — AQI SUMMARY
# Percentage breakdown of AQI categories per city
# Shows how often each city is in Good / Moderate / Hazardous
# ============================================================

aqi_summary_df = spark.sql(f"""
    SELECT
        city,
        aqi_category,
        COUNT(*)                                            AS reading_count,
        ROUND(COUNT(*) * 100.0 /
            SUM(COUNT(*)) OVER (PARTITION BY city), 2)     AS percentage,
        CURRENT_TIMESTAMP()                                 AS last_updated
    FROM {SILVER_TABLE}
    WHERE pollutant     = 'pm25'
    AND   is_valid      = true
    AND   aqi_category  != 'N/A'
    GROUP BY city, aqi_category
    ORDER BY city, percentage DESC
""")

#aqi_summary_df.cache()

print(f"✅ AQI summary aggregation complete.")
print(f"   Rows generated : {aqi_summary_df.count()}")
print(f"\n📊 Preview — AQI Category Breakdown:")
aqi_summary_df.show(30, truncate=False)

✅ AQI summary aggregation complete.
   Rows generated : 60

📊 Preview — AQI Category Breakdown:
+-------+---------------------+-------------+----------+--------------------------+
|city   |aqi_category         |reading_count|percentage|last_updated              |
+-------+---------------------+-------------+----------+--------------------------+
|Beijing|Good                 |49           |49.00     |2026-02-19 20:10:35.053583|
|Beijing|Moderate             |48           |48.00     |2026-02-19 20:10:35.053583|
|Beijing|Unhealthy (Sensitive)|2            |2.00      |2026-02-19 20:10:35.053583|
|Beijing|Unhealthy            |1            |1.00      |2026-02-19 20:10:35.053583|
|Cairo  |Unhealthy            |24           |48.00     |2026-02-19 20:10:35.053583|
|Cairo  |Moderate             |14           |28.00     |2026-02-19 20:10:35.053583|
|Cairo  |Unhealthy (Sensitive)|12           |24.00     |2026-02-19 20:10:35.053583|
|Delhi  |Hazardous            |30           |60.00     |2026-02-

In [0]:
# ============================================================
# WRITE ALL THREE GOLD TABLES TO DELTA
# ============================================================

# Gold Table 1 — City Rankings
(
    city_rankings_final_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(GOLD_TABLE_CITY)
)
print(f"✅ Written : {GOLD_TABLE_CITY}")

# Gold Table 2 — Pollutant Trends
(
    trends_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(GOLD_TABLE_TREND)
)
print(f"✅ Written : {GOLD_TABLE_TREND}")

# Gold Table 3 — AQI Summary
(
    aqi_summary_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(GOLD_TABLE_AQI)
)
print(f"✅ Written : {GOLD_TABLE_AQI}")

print(f"\n🥇 All three Gold tables written successfully.")

✅ Written : air_quality_db.gold_city_rankings
✅ Written : air_quality_db.gold_pollutant_trends
✅ Written : air_quality_db.gold_aqi_summary

🥇 All three Gold tables written successfully.


In [0]:
# ============================================================
# VERIFY ALL GOLD TABLES
# ============================================================

city_count  = spark.sql(f"SELECT COUNT(*) AS cnt FROM {GOLD_TABLE_CITY}").collect()[0]["cnt"]
trend_count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {GOLD_TABLE_TREND}").collect()[0]["cnt"]
aqi_count   = spark.sql(f"SELECT COUNT(*) AS cnt FROM {GOLD_TABLE_AQI}").collect()[0]["cnt"]

print("=" * 55)
print("  🥇 GOLD AGGREGATION — SUMMARY REPORT")
print("=" * 55)
print(f"  Silver valid records used  : {silver_count}")
print(f"  gold_city_rankings rows    : {city_count}")
print(f"  gold_pollutant_trends rows : {trend_count}")
print(f"  gold_aqi_summary rows      : {aqi_count}")
print("=" * 55)

print(f"\n📊 Top 5 Most Polluted Cities (PM2.5):")
spark.sql(f"""
    SELECT city, avg_value AS avg_pm25, dominant_aqi
    FROM {GOLD_TABLE_CITY}
    WHERE pollutant = 'pm25'
    ORDER BY avg_value DESC
    LIMIT 5
""").show(truncate=False)

print(f"\n📊 Top 5 Cleanest Cities (PM2.5):")
spark.sql(f"""
    SELECT city, avg_value AS avg_pm25, dominant_aqi
    FROM {GOLD_TABLE_CITY}
    WHERE pollutant = 'pm25'
    ORDER BY avg_value ASC
    LIMIT 5
""").show(truncate=False)

print(f"\n▶️  Next Step: Open and run  04_visualization")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8945000014223545>, line 5
      1 # ============================================================
      2 # VERIFY ALL GOLD TABLES
      3 # ============================================================
----> 5 city_count  = spark.sql(f"SELECT COUNT(*) AS cnt FROM {GOLD_TABLE_CITY}").collect()[0]["cnt"]
      6 trend_count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {GOLD_TABLE_TREND}").collect()[0]["cnt"]
      7 aqi_count   = spark.sql(f"SELECT COUNT(*) AS cnt FROM {GOLD_TABLE_AQI}").collect()[0]["cnt"]

NameError: name 'GOLD_TABLE_CITY' is not defined